# Toronto data

1. [Importing packages](#import)
2. [Loading data](#load) <br>
    2.1 [Loading data using pandas](#loadpd) <br>
    2.2 [Loading using requests and beautifulSoup](#loadbs4)
3. [Data Preparation](#dataprep)
4. [Export](#export)

### <a id="import"> </a> Importing packages

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [4]:
# url where the needed table is located
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### <a id="load"> </a> Loading data
Two methods are available. Using pandas built in *read_html* method or using using *requests* and *BeautifulSoup*.
The one retained for the notebook is loading using *pandas* if want to change the method just uncomment the [Loading using requests and beautifulSoup](#loadbs4) section

#### <a id="loadpd"> </a> Loading data using pandas

In [5]:
raw_table = pd.read_html(url_wiki)[0]

#### <a id="loadbs4"> </a> Loading using requests and beautifulSoup

In [86]:
# results = requests.get(url_wiki)
# print(results)

In [87]:
# soup = BeautifulSoup(results.text)

# table = soup.table.tbody

# rows = table.find_all('tr')
# columns = []
# data =[]

# columns = [th.text.strip() for th in table.find_all('th')]


# for row in rows:
#     col = row.find_all('td')
#     col = [td.text.strip() for td in col]
#     data.append([td for td in col if td])

# data.pop(0)

# raw_table = pd.DataFrame(data, columns=columns)
# raw_table.head()

In [7]:
print(raw_table.shape)
raw_table.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### <a id="dataprep"> </a> Data preparation

We will remove rows with not assigned Boroughs

In [8]:
raw_table = raw_table[raw_table['Borough']!='Not assigned']
print(raw_table.shape)
raw_table.head()

(211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


This cell will group each duplicate postcode and append corresponding neighbourhoods into list and create a list with all those values

In [19]:
res = []
for i in raw_table[raw_table.duplicated('Postcode', keep=False)].index:
    ind = i
    postcode = raw_table.loc[i]['Postcode']
    borough = raw_table.loc[i]['Borough']
    neigh = raw_table.loc[i]['Neighbourhood']
    row = [ind, postcode,borough,neigh]
    if not res:
        res.append(row)
    elif postcode == res[-1][1]:
        res[-1][3] = res[-1][3] + ", " + neigh
    else:
        res.append(row)

Transform the previous list into dataframe 

In [20]:
dup_values = pd.DataFrame(res, columns=['Index','Postcode','Borough','Neighbourhood'])
dup_values.set_index('Index', inplace=True)
print(dup_values.shape)
dup_values.head()

(57, 3)


,Postcode,Borough,Neighbourhood
Index,,,
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
11,M1B,Scarborough,"Rouge, Malvern"
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


Create a dataframe with only postcodes having unique values in df

In [21]:
unique_table = raw_table[raw_table.duplicated('Postcode', keep=False) ==False]
print(unique_table.shape)
unique_table.head()

(46, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
14,M3B,North York,Don Mills North


Merge two df into one

In [22]:
cleaned_table = pd.concat([unique_table, dup_values])
cleaned_table.sort_index(inplace=True)
cleaned_table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned


Put value of 'Borough' cell into cells with 'Not assigned' for neighbourhood

In [23]:
for i in cleaned_table.index:
    if cleaned_table.loc[i]['Neighbourhood']=='Not assigned':
        cleaned_table.loc[i]['Neighbourhood'] = cleaned_table.loc[i]['Borough']
        

Print the shape of the resulting df

In [24]:
cleaned_table.shape

(103, 3)

In [25]:
cleaned_table.rename({'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, axis=1, inplace=True)
cleaned_table.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


### <a id="export"> </a> Export to csv file.

In [27]:
cleaned_table.to_csv('data/toronto_wiki_data')

{'file_name': 'toronto_wiki_data',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneproject-donotdelete-pr-cmubpqoqshzron',
 'asset_id': 'a90c3e1e-7ecc-4736-a8f8-2f84c835246a'}